In [18]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [19]:
gmsh.open("buckling-3.geo")

Info    : Reading 'buckling-3.geo'...
Info    : Meshing 1D...                                                                                                     
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Line)
Info    : [ 90%] Meshing curve 17 (Line)
Info    : [ 90%] Meshing curve 18 (Line)
Info    : [100%] Meshing curve 19 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D 

In [20]:
mat = FEM.material("body", E=2.0e5, ν=0.3)
problem = FEM.Problem([mat], type="Solid", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("buckling-3", "Solid", 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 775)

In [21]:
l = gmsh.parser.getNumber("l")[1]
a = gmsh.parser.getNumber("a")[1]
F = -43275 + 94

-43181

In [22]:
Fy(x, y, z) = -100z / 10

Fy (generic function with 1 method)

In [23]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0, uz=0)
supp0 = FEM.displacementConstraint("left", ux=0, uy=0, uz=0)
supp1 = FEM.displacementConstraint("load", uy=0, uz=0)
supp2 = FEM.displacementConstraint("load", ux=0, uy=0, uz=0)
load = FEM.load("load", fx=F / a^2)
load0 = FEM.load("right", fx=F / a^2)
loadBand = FEM.load("right", fy=Fy)

("right", 0, Main.Fy, 0)

In [24]:
B = FEM.solveBuckling(problem, [load0], [supp0])

LowLevelFEM.Eigen([1.0000134518454247, 1.0000134518462185, 8.488058737572048, 8.48805873757483, 21.240793701882094, 21.24079370188233], [-2.4245516570032817e-17 -2.2572979727273432e-17 … -1.1855677905805616e-17 2.915503628083669e-18; -1.839243105281654e-17 -1.7179950987969113e-17 … -7.054093571835156e-19 3.650825681921301e-18; … ; 0.06872677421753781 0.06300438961033106 … 0.023821248134359912 -0.017020935737640602; 0.01313116309477728 -0.03042925478960272 … 0.0008463689460581035 0.016552104495186917])

In [25]:
B.f

6-element Vector{Float64}:
  1.0000134518454247
  1.0000134518462185
  8.488058737572048
  8.48805873757483
 21.240793701882094
 21.24079370188233

In [26]:
bk = FEM.showBucklingResults(problem, B, visible=false)

0

In [27]:
M = FEM.solveModalAnalysis(problem, load=[load0], bc=[supp0], n=10, fₘᵢₙ=1.01)

LowLevelFEM.Eigen([3.2008237473921834, 3.2008244100976033, 4550.402122808056, 4550.402125012284, 6942.794620770964, 12658.088891621863, 12780.99080740868, 12780.9908130363, 20811.083894084983, 23508.291759058502], [1.2530302262992076e-19 7.052498345489531e-19 … -2.372154946732958e-18 -3.940082359254047e-18; -5.1854961460719077e-20 -2.8319470579759245e-19 … 2.26966207468516e-19 3.339747008743278e-19; … ; 0.06961978273423193 -0.013650179359273453 … 0.02508597864821466 0.03756792027903171; 0.006987982212164928 0.0686240851371404 … -0.025085978596522618 0.00939820422797368])

In [28]:
ma = FEM.showModalResults(problem, M, visible=false)

1

In [29]:
gmsh.view.option.setNumber(bk, "DisplacementFactor", 300)
gmsh.view.option.setNumber(ma, "DisplacementFactor", 300)

In [30]:
E = problem.material[1].E
A = a * a
I = a^4 / 12
i = √(I / A)
β = 2
l0 = β * l
λ = l0 / i
display("λ = $λ")
σₖᵣ = π^2 * E / λ^2
Fₖᵣ = σₖᵣ * A
display("theory: Fₖᵣ = $Fₖᵣ")

Fkrfem = -F * B.f[1]
display("fem: Fₖᵣ = $Fkrfem")

"λ = 69.28203230275508"

"theory: Fₖᵣ = 41123.351671205666"

"fem: Fₖᵣ = 43181.580864137286"

In [31]:
n = 1
ρ = problem.material[1].ρ
ω0 = π / 2 / l * √(E / ρ) / 2π
ω0i = n^2 * π^2 / l^2 * √(E * I / (ρ * A)) / 2π
ωi = n^2 * π^2 / l^2 * √(E * I / (ρ * A)) * √(1 + l^2 * 0F / (n^2 * π^2 * E * I)) / 2π
display("ω0 = $ω0")
display("i = $n")
display("ω0i = $ω0i")
display("ωi = $ωi")
display(M.f)

"ω0 = 12618.86162812672"

"i = 1"

"ω0i = 2288.808319845581"

"ωi = 2288.808319845581"

10-element Vector{Float64}:
     3.2008237473921834
     3.2008244100976033
  4550.402122808056
  4550.402125012284
  6942.794620770964
 12658.088891621863
 12780.99080740868
 12780.9908130363
 20811.083894084983
 23508.291759058502

In [32]:
K = FEM.stiffnessMatrix(problem)
f = FEM.loadVector(problem, [loadBand])
FEM.applyBoundaryConditions!(problem, K, f, [supp0])
q = FEM.solveDisplacement(K, f)

err = 1
counter = 0

while err > 1e-8 && counter < 20
    counter += 1
    q0 = copy(q)
    f = FEM.loadVector(problem, [loadBand])
    K = FEM.stiffnessMatrix(problem)
    Knl = FEM.nonLinearStiffnessMatrix(problem, q)
    FEM.applyBoundaryConditions!(problem, K, Knl, f, [supp0])
    q = FEM.solveDisplacement(K + Knl, f)
    err = sum(abs, q - q0) / sum(abs, q0)
    display("err($counter) = $err")
end

u = FEM.showDoFResults(problem, q, "uvec", visible=true)
gmsh.view.option.setNumber(u, "DisplacementFactor", 1)
gmsh.view.option.setNumber(u, "VectorType", 5)

"err(1) = 0.00037964263030337787"

"err(2) = 0.0034178664468301843"

"err(3) = 2.4051050870009733e-5"

"err(4) = 2.8462974824623095e-5"

"err(5) = 4.1372501998703386e-7"

"err(6) = 3.131090964132681e-7"

"err(7) = 6.783928119666768e-9"

In [33]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [34]:
gmsh.finalize()